In [ ]:
# import os
# os.chdir(r"C:\Users\Jahid Chowdhury\OneDrive - Kansas State University\PhD Research\Anytime Control Project (2023)\Anytime tool")

In [14]:
# We first import the time module to compute runtime
import time
start_time = time.time()

# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    return Box

# Minkowski sum function
def minkowski_pol(P1, P2):
    V1 = P1.vertices_list()
    V2 = P2.vertices_list()
    V3 = []
    for v1 in V1:
        for v2 in V2:
            v = [a+b for a,b in zip(v1,v2)]
            V3.append(v)
    print(V3)
    P_out = Polyhedron(vertices = V3, backend='ppl', base_ring=QQ)
    return P_out


# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 1: ")
exp_num = v
# Evaluate the inputs
if (int(v)==1):
    input_file = open(r"inputs/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Create the box
v = [0]*n
r = [ep]*n
Box = make_box(v,r)

# Opening a file to write the outputs
exp_string = "outputs/alg1/ex" + exp_num + ".csv"
f = open(exp_string, "a")
f.write('\nNew experiment:\n')


f.write('Iteration, Num_Cons(Xout), Num_Ver(Xout), Volume(Xout), Runtime\n')
# Running algorithm 1
Xc = X0
Uc = U0
iter = 1
error = 0
for i in range(iter):
    try:
        # print('iteration = ', i+1, file=f)
        AXc = A*Xc
        # print("AXc = ", AXc, file=f)
        BUc = B*Uc
        # print("BUc = ", BUc, file=f)
        Xc = AXc + BUc
        # print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
        # print('Box = ', Box)
        Xp = minkowski_pol(Xc, Box)
        # print('Number_of_Cons(Xp) = ', len(list(Xp.Hrepresentation())), file=f)
        Uc = K*Xp
        # print('Number_of_Cons(Uc) = ', len(list(Uc.Hrepresentation())), file=f)
        # print('Vol(Xc) = ', float(Xc.volume()),  file=f)
        # print('Vol(Xp) = ', float(Xp.volume()),  file=f)
        # print('Vol(Uc) = ', float(Uc.volume()),  file=f)
        curr_time = (time.time()-start_time)*10**3
        msg = str((i+1, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), "{:e}".format(float(Xc.volume())), curr_time))
        msg = msg[1:-1]
        print(msg)
        f.write(msg+'\n')
    except ValueError as ve:
        print('ValueError = ', ve, file=f)
        error = ve
        break
    except ZeroDivisionError as ze:
        # print("ZeroDivisionError = ", ze, file=f)
        msg = str((i+1, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), "{:e}".format(float(Xc.volume())), curr_time))
        msg = msg[1:-1]
        print(msg)
        f.write(msg+'\n')
        error = ze


if error:
    error_msg = "Failed at iteration : "+str(i)
else:
    error_msg = "Pass"

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3

# msg = str((iter, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), len(list(Xc.vertices())[0]), total_time_in_ms, error_msg))
# msg = msg[1:-1]
# f.write(msg)

print("\nRuntime: ", total_time_in_ms, file = f)

if error:
    print('\n', error_msg, file=f)


# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg1/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc


[[0.20020000000000002, -0.1998, -0.19970000000000002, 0.20006666666667, -0.19993333333333002, 0.2], [0.20020000000000002, 0.20020000000000002, -0.19970000000000002, -0.19993333333333002, -0.19993333333333002, -0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, -0.19993333333333002, 0.20006666666667, 0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, 0.20006666666667, -0.19993333333333002, -0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, 0.20006666666667, 0.20006666666667, -0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, 0.20006666666667, 0.20006666666667, 0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, 0.20006666666667, -0.19993333333333002, 0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, -0.19993333333333002, -0.19993333333333002, 0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, -0.19993333333333002, -0.19993333333333002, -0.2], [0.20020000000000002, 0.20020000000000002, 0.2003, -0.19993333333333002, 0.20006666666667, -0.2], [0.2

/private/var/tmp/sage-10.1-current/local/var/lib/sage/venv-python3.11.1/lib/python3.11/site-packages/sage/geometry/polyhedron/backend_cdd_rdf.py:188: UserWarning: This polyhedron data is numerically complicated; cdd could not convert between the inexact V and H representation without loss of data. The resulting object might show inconsistencies.
  warn("This polyhedron data is numerically complicated; cdd could not convert between the inexact V and H representation without loss of data. The resulting object might show inconsistencies.")


5, 9, 8, '0.000000e+00', 4453.449964523315
[[0.2, -0.2, -0.2, 0.19989222507930002, -0.19989222507930002, 0.198483665866], [0.2, 0.2, -0.2, -0.2001077749207, -0.19989222507930002, -0.20151633413400002], [0.2, 0.2, 0.2, -0.2001077749207, 0.2001077749207, 0.198483665866], [0.2, 0.2, 0.2, 0.19989222507930002, -0.19989222507930002, -0.20151633413400002], [0.2, 0.2, 0.2, 0.19989222507930002, 0.2001077749207, -0.20151633413400002], [0.2, 0.2, 0.2, 0.19989222507930002, 0.2001077749207, 0.198483665866], [0.2, 0.2, 0.2, 0.19989222507930002, -0.19989222507930002, 0.198483665866], [0.2, 0.2, 0.2, -0.2001077749207, -0.19989222507930002, 0.198483665866], [0.2, 0.2, 0.2, -0.2001077749207, -0.19989222507930002, -0.20151633413400002], [0.2, 0.2, 0.2, -0.2001077749207, 0.2001077749207, -0.20151633413400002], [0.2, 0.2, -0.2, -0.2001077749207, 0.2001077749207, -0.20151633413400002], [0.2, 0.2, -0.2, -0.2001077749207, 0.2001077749207, 0.198483665866], [0.2, 0.2, -0.2, 0.19989222507930002, -0.1998922250793

A 3-dimensional polyhedron in RDF^6 defined as the convex hull of 8 vertices (use the .plot() method to plot)